# 4. 位置编码 (Positional Encoding)

在前面的教程中，我们学习了注意力机制和多头注意力。但是这些机制有一个重要的局限性：**它们无法感知序列中元素的位置信息**。位置编码就是为了解决这个问题而设计的。

## 4.1 为什么需要位置编码？

### 注意力机制的位置盲区

自注意力机制本质上是一个**置换不变**的操作，这意味着：
- 如果我们打乱输入序列的顺序，注意力权重的模式会保持不变
- 模型无法区分 "我爱你" 和 "你爱我" 这样的句子
- 在很多NLP任务中，词序是极其重要的语法和语义信息

### 位置信息的重要性

考虑以下例子：
1. **"The cat sat on the mat"** vs **"The mat sat on the cat"**
2. **"John loves Mary"** vs **"Mary loves John"**
3. **"Not good"** vs **"Good not"**

显然，位置信息对于理解语言的含义至关重要。

![位置编码示意图](images/positional_encoding.png)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from typing import Optional

# 设置随机种子和图表样式
torch.manual_seed(42)
np.random.seed(42)
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print(f"PyTorch版本: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

## 4.2 演示注意力的位置不变性

让我们首先通过实验证明注意力机制确实是位置不变的：

In [ ]:
# 从前面的教程导入多头注意力
class SimpleMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(SimpleMultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.W_o(output)
        
        return output, attention_weights

# 创建测试数据
d_model = 32
seq_len = 5
batch_size = 1

# 原始序列
original_seq = torch.randn(batch_size, seq_len, d_model)
print(f"原始序列形状: {original_seq.shape}")

# 创建一个置换（打乱顺序）
permutation = torch.randperm(seq_len)
permuted_seq = original_seq[:, permutation, :]
print(f"置换索引: {permutation.tolist()}")
print(f"置换后序列形状: {permuted_seq.shape}")

# 测试注意力机制
attention = SimpleMultiHeadAttention(d_model, num_heads=4)

# 对原始序列和置换序列应用注意力
original_output, original_weights = attention(original_seq)
permuted_output, permuted_weights = attention(permuted_seq)

print(f"\n原始序列注意力权重形状: {original_weights.shape}")
print(f"置换序列注意力权重形状: {permuted_weights.shape}")

# 检查注意力权重的置换不变性
# 我们需要按照置换顺序重新排列权重矩阵进行比较
reordered_weights = permuted_weights[0, 0, permutation, :][:, permutation]
original_weights_first_head = original_weights[0, 0]

print(f"\n注意力权重的差异（应该很小）: {torch.mean(torch.abs(reordered_weights - original_weights_first_head)):.6f}")

## 4.3 正弦位置编码 (Sinusoidal Positional Encoding)

Transformer论文中提出的经典位置编码使用正弦和余弦函数：

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

其中：
- $pos$ 是位置索引
- $i$ 是维度索引
- $d_{model}$ 是模型维度

### 正弦位置编码的优点：
1. **唯一性**：每个位置都有唯一的编码
2. **相对位置**：模型可以学习相对位置关系
3. **外推性**：可以处理训练时未见过的序列长度
4. **确定性**：不需要学习参数

In [ ]:
class SinusoidalPositionalEncoding(nn.Module):
    """
    正弦位置编码的实现
    """
    def __init__(self, d_model: int, max_seq_len: int = 5000):
        super(SinusoidalPositionalEncoding, self).__init__()
        
        self.d_model = d_model
        
        # 创建位置编码矩阵
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        
        # 计算分母项
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                            (-math.log(10000.0) / d_model))
        
        # 计算正弦和余弦
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数维度使用sin
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数维度使用cos
        
        # 添加batch维度并注册为buffer（不会被当作参数）
        pe = pe.unsqueeze(0)  # [1, max_seq_len, d_model]
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: [batch_size, seq_len, d_model]
        Returns:
            x + positional_encoding: [batch_size, seq_len, d_model]
        """
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]
    
    def get_encoding(self, seq_len: int) -> torch.Tensor:
        """
        获取指定长度的位置编码
        """
        return self.pe[:, :seq_len, :]

# 创建位置编码实例
d_model = 64
max_seq_len = 100
pos_encoding = SinusoidalPositionalEncoding(d_model, max_seq_len)

# 获取前20个位置的编码用于可视化
seq_len_viz = 20
pe_matrix = pos_encoding.get_encoding(seq_len_viz)
print(f"位置编码矩阵形状: {pe_matrix.shape}")
print(f"位置编码数值范围: [{pe_matrix.min():.3f}, {pe_matrix.max():.3f}]")

## 4.4 可视化位置编码

让我们可视化位置编码，理解其模式和特性：

In [ ]:
def visualize_positional_encoding(pos_encoding, seq_len=50, d_model=64):
    """
    可视化位置编码
    """
    # 获取位置编码
    pe = pos_encoding.get_encoding(seq_len)[0].numpy()  # [seq_len, d_model]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. 位置编码热力图
    im1 = axes[0, 0].imshow(pe.T, cmap='RdBu_r', aspect='auto')
    axes[0, 0].set_title('位置编码热力图')
    axes[0, 0].set_xlabel('位置')
    axes[0, 0].set_ylabel('维度')
    plt.colorbar(im1, ax=axes[0, 0])
    
    # 2. 选择几个维度展示波形
    positions = np.arange(seq_len)
    selected_dims = [0, 1, 2, 3, 10, 20]  # 选择几个维度
    
    for i, dim in enumerate(selected_dims):
        if dim < d_model:
            axes[0, 1].plot(positions, pe[:, dim], label=f'维度 {dim}', alpha=0.7)
    
    axes[0, 1].set_title('不同维度的位置编码波形')
    axes[0, 1].set_xlabel('位置')
    axes[0, 1].set_ylabel('编码值')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 不同位置的编码分布
    selected_positions = [0, 5, 10, 15, 20]
    
    for pos in selected_positions:
        if pos < seq_len:
            axes[1, 0].plot(pe[pos, :], label=f'位置 {pos}', alpha=0.7)
    
    axes[1, 0].set_title('不同位置的编码向量')
    axes[1, 0].set_xlabel('维度')
    axes[1, 0].set_ylabel('编码值')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 位置编码的频率分析
    # 计算不同维度的频率
    div_terms = [10000 ** (2 * i / d_model) for i in range(d_model // 2)]
    frequencies = [1 / term for term in div_terms]
    
    axes[1, 1].semilogy(frequencies[:min(32, len(frequencies))], 'o-')
    axes[1, 1].set_title('位置编码的频率谱')
    axes[1, 1].set_xlabel('维度对 (i)')
    axes[1, 1].set_ylabel('频率 (log scale)')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return pe

# 可视化位置编码
pe_matrix = visualize_positional_encoding(pos_encoding, seq_len=50, d_model=64)

## 4.5 位置编码的相对位置特性

正弦位置编码的一个重要特性是它可以表示相对位置关系。让我们验证这个特性：

In [ ]:
def analyze_relative_positions(pos_encoding, max_seq_len=30):
    """
    分析位置编码的相对位置特性
    """
    # 获取位置编码
    pe = pos_encoding.get_encoding(max_seq_len)[0]  # [seq_len, d_model]
    
    # 计算所有位置对之间的相似性
    similarity_matrix = torch.zeros(max_seq_len, max_seq_len)
    
    for i in range(max_seq_len):
        for j in range(max_seq_len):
            # 使用余弦相似度
            cos_sim = F.cosine_similarity(pe[i:i+1], pe[j:j+1], dim=1)
            similarity_matrix[i, j] = cos_sim
    
    # 可视化相似性矩阵
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 位置相似性热力图
    im1 = ax1.imshow(similarity_matrix.numpy(), cmap='RdBu_r')
    ax1.set_title('位置编码相似性矩阵')
    ax1.set_xlabel('位置 j')
    ax1.set_ylabel('位置 i')
    plt.colorbar(im1, ax=ax1)
    
    # 分析相对距离与相似性的关系
    distances = []
    similarities = []
    
    for i in range(max_seq_len):
        for j in range(max_seq_len):
            distance = abs(i - j)
            similarity = similarity_matrix[i, j].item()
            distances.append(distance)
            similarities.append(similarity)
    
    # 按距离分组计算平均相似性
    max_distance = max(distances)
    avg_similarities = []
    distance_range = range(max_distance + 1)
    
    for d in distance_range:
        same_distance_sims = [sim for dist, sim in zip(distances, similarities) if dist == d]
        avg_similarities.append(np.mean(same_distance_sims))
    
    ax2.plot(distance_range, avg_similarities, 'o-', linewidth=2, markersize=4)
    ax2.set_title('相对距离 vs 位置编码相似性')
    ax2.set_xlabel('相对距离')
    ax2.set_ylabel('平均余弦相似性')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"位置0和位置1的相似性: {similarity_matrix[0, 1]:.4f}")
    print(f"位置0和位置5的相似性: {similarity_matrix[0, 5]:.4f}")
    print(f"位置5和位置6的相似性: {similarity_matrix[5, 6]:.4f}")
    print(f"位置0和位置15的相似性: {similarity_matrix[0, 15]:.4f}")
    
    return similarity_matrix

# 分析相对位置特性
similarity_matrix = analyze_relative_positions(pos_encoding, max_seq_len=20)

## 4.6 可学习位置编码 (Learned Positional Encoding)

除了固定的正弦位置编码，我们还可以使用可学习的位置编码：

In [ ]:
class LearnedPositionalEncoding(nn.Module):
    """
    可学习的位置编码
    """
    def __init__(self, d_model: int, max_seq_len: int = 5000):
        super(LearnedPositionalEncoding, self).__init__()
        
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # 创建可学习的位置嵌入
        self.position_embeddings = nn.Embedding(max_seq_len, d_model)
        
        # 初始化
        nn.init.normal_(self.position_embeddings.weight, mean=0, std=0.1)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: [batch_size, seq_len, d_model]
        Returns:
            x + positional_encoding: [batch_size, seq_len, d_model]
        """
        batch_size, seq_len, _ = x.size()
        
        # 创建位置索引
        positions = torch.arange(0, seq_len, device=x.device, dtype=torch.long)
        positions = positions.unsqueeze(0).expand(batch_size, -1)  # [batch_size, seq_len]
        
        # 获取位置编码
        position_encodings = self.position_embeddings(positions)  # [batch_size, seq_len, d_model]
        
        return x + position_encodings
    
    def get_encoding(self, seq_len: int) -> torch.Tensor:
        """
        获取指定长度的位置编码
        """
        positions = torch.arange(0, seq_len, dtype=torch.long)
        return self.position_embeddings(positions).unsqueeze(0)

class RelativePositionalEncoding(nn.Module):
    """
    相对位置编码（T5风格）
    """
    def __init__(self, d_model: int, max_relative_distance: int = 32):
        super(RelativePositionalEncoding, self).__init__()
        
        self.d_model = d_model
        self.max_relative_distance = max_relative_distance
        
        # 创建相对位置嵌入
        vocab_size = 2 * max_relative_distance + 1  # -max_dist 到 +max_dist
        self.relative_embeddings = nn.Embedding(vocab_size, d_model)
    
    def forward(self, seq_len: int) -> torch.Tensor:
        """
        计算相对位置编码矩阵
        
        Args:
            seq_len: 序列长度
        
        Returns:
            relative_position_encoding: [seq_len, seq_len, d_model]
        """
        # 创建相对位置矩阵
        positions = torch.arange(seq_len, dtype=torch.long)
        relative_positions = positions.unsqueeze(1) - positions.unsqueeze(0)  # [seq_len, seq_len]
        
        # 截断相对距离
        relative_positions = torch.clamp(
            relative_positions, 
            -self.max_relative_distance, 
            self.max_relative_distance
        )
        
        # 转换为正索引
        relative_positions += self.max_relative_distance
        
        # 获取相对位置编码
        relative_encodings = self.relative_embeddings(relative_positions)
        
        return relative_encodings

# 创建不同类型的位置编码
d_model = 32
seq_len = 15

# 正弦位置编码
sin_pos_enc = SinusoidalPositionalEncoding(d_model)

# 可学习位置编码
learned_pos_enc = LearnedPositionalEncoding(d_model)

# 相对位置编码
relative_pos_enc = RelativePositionalEncoding(d_model)

# 获取编码
sin_encoding = sin_pos_enc.get_encoding(seq_len)[0]
learned_encoding = learned_pos_enc.get_encoding(seq_len)[0]
relative_encoding = relative_pos_enc(seq_len)

print(f"正弦位置编码形状: {sin_encoding.shape}")
print(f"可学习位置编码形状: {learned_encoding.shape}")
print(f"相对位置编码形状: {relative_encoding.shape}")

print(f"\n各类编码的参数量:")
print(f"正弦位置编码: 0 (无参数)")
print(f"可学习位置编码: {sum(p.numel() for p in learned_pos_enc.parameters()):,}")
print(f"相对位置编码: {sum(p.numel() for p in relative_pos_enc.parameters()):,}")

## 4.7 对比不同位置编码方法

让我们对比不同位置编码方法的效果：

In [ ]:
def compare_positional_encodings():
    """
    比较不同的位置编码方法
    """
    d_model = 64
    seq_len = 20
    
    # 创建不同的位置编码
    sin_pe = SinusoidalPositionalEncoding(d_model)
    learned_pe = LearnedPositionalEncoding(d_model)
    
    # 获取编码
    sin_encoding = sin_pe.get_encoding(seq_len)[0].detach().numpy()
    learned_encoding = learned_pe.get_encoding(seq_len)[0].detach().numpy()
    
    # 可视化对比
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # 正弦位置编码热力图
    im1 = axes[0, 0].imshow(sin_encoding.T, cmap='RdBu_r', aspect='auto')
    axes[0, 0].set_title('正弦位置编码')
    axes[0, 0].set_xlabel('位置')
    axes[0, 0].set_ylabel('维度')
    plt.colorbar(im1, ax=axes[0, 0])
    
    # 可学习位置编码热力图
    im2 = axes[0, 1].imshow(learned_encoding.T, cmap='RdBu_r', aspect='auto')
    axes[0, 1].set_title('可学习位置编码')
    axes[0, 1].set_xlabel('位置')
    axes[0, 1].set_ylabel('维度')
    plt.colorbar(im2, ax=axes[0, 1])
    
    # 编码差异
    diff = learned_encoding - sin_encoding
    im3 = axes[0, 2].imshow(diff.T, cmap='RdBu_r', aspect='auto')
    axes[0, 2].set_title('差异 (可学习 - 正弦)')
    axes[0, 2].set_xlabel('位置')
    axes[0, 2].set_ylabel('维度')
    plt.colorbar(im3, ax=axes[0, 2])
    
    # 选择几个位置的编码向量进行比较
    positions_to_show = [0, 5, 10, 15]
    
    for pos in positions_to_show:
        axes[1, 0].plot(sin_encoding[pos, :], label=f'位置 {pos}', alpha=0.7)
    axes[1, 0].set_title('正弦编码 - 不同位置的向量')
    axes[1, 0].set_xlabel('维度')
    axes[1, 0].set_ylabel('编码值')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    for pos in positions_to_show:
        axes[1, 1].plot(learned_encoding[pos, :], label=f'位置 {pos}', alpha=0.7)
    axes[1, 1].set_title('可学习编码 - 不同位置的向量')
    axes[1, 1].set_xlabel('维度')
    axes[1, 1].set_ylabel('编码值')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # 统计特性比较
    sin_std = np.std(sin_encoding, axis=1)
    learned_std = np.std(learned_encoding, axis=1)
    
    axes[1, 2].plot(sin_std, label='正弦编码标准差', marker='o')
    axes[1, 2].plot(learned_std, label='可学习编码标准差', marker='s')
    axes[1, 2].set_title('不同位置编码的变异性')
    axes[1, 2].set_xlabel('位置')
    axes[1, 2].set_ylabel('标准差')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 打印统计信息
    print("位置编码统计比较:")
    print(f"正弦编码 - 均值: {np.mean(sin_encoding):.4f}, 标准差: {np.std(sin_encoding):.4f}")
    print(f"可学习编码 - 均值: {np.mean(learned_encoding):.4f}, 标准差: {np.std(learned_encoding):.4f}")
    print(f"编码差异的均值: {np.mean(np.abs(diff)):.4f}")

# 运行比较
compare_positional_encodings()

## 4.8 位置编码对注意力的影响

让我们实验位置编码如何影响注意力模式：

In [ ]:
def demonstrate_positional_encoding_effect():
    """
    演示位置编码对注意力模式的影响
    """
    d_model = 64
    num_heads = 4
    seq_len = 8
    batch_size = 1
    
    # 创建输入数据（相同的内容，不同的位置）
    # 为了演示效果，我们创建一个重复的模式
    base_vector = torch.randn(1, 1, d_model)
    repeated_input = base_vector.repeat(batch_size, seq_len, 1)
    print(f"输入形状: {repeated_input.shape}")
    print("注意：输入内容在所有位置都相同，只有位置不同")
    
    # 创建注意力层
    attention = SimpleMultiHeadAttention(d_model, num_heads)
    
    # 创建位置编码
    pos_encoding = SinusoidalPositionalEncoding(d_model)
    
    # 1. 没有位置编码的注意力
    output_no_pos, weights_no_pos = attention(repeated_input)
    
    # 2. 有位置编码的注意力
    input_with_pos = pos_encoding(repeated_input)
    output_with_pos, weights_with_pos = attention(input_with_pos)
    
    # 可视化对比
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # 显示前4个头的注意力权重
    for head in range(4):
        # 无位置编码
        im1 = axes[0, head].imshow(weights_no_pos[0, head].detach().numpy(), 
                                  cmap='Blues', vmin=0, vmax=1)
        axes[0, head].set_title(f'无位置编码 - 头 {head+1}')
        axes[0, head].set_xlabel('Key位置')
        axes[0, head].set_ylabel('Query位置')
        
        # 添加数值标注
        weights_np = weights_no_pos[0, head].detach().numpy()
        for i in range(seq_len):
            for j in range(seq_len):
                axes[0, head].text(j, i, f'{weights_np[i, j]:.2f}', 
                                  ha='center', va='center', fontsize=8)
        
        # 有位置编码
        im2 = axes[1, head].imshow(weights_with_pos[0, head].detach().numpy(), 
                                  cmap='Blues', vmin=0, vmax=1)
        axes[1, head].set_title(f'有位置编码 - 头 {head+1}')
        axes[1, head].set_xlabel('Key位置')
        axes[1, head].set_ylabel('Query位置')
        
        # 添加数值标注
        weights_np = weights_with_pos[0, head].detach().numpy()
        for i in range(seq_len):
            for j in range(seq_len):
                axes[1, head].text(j, i, f'{weights_np[i, j]:.2f}', 
                                  ha='center', va='center', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # 分析注意力权重的差异
    print("\n注意力权重分析:")
    
    # 计算注意力权重的熵（衡量分散程度）
    def calculate_entropy(weights):
        # weights: [batch, heads, seq_len, seq_len]
        weights_np = weights[0].detach().numpy()
        entropies = []
        for head in range(weights_np.shape[0]):
            head_entropy = []
            for i in range(weights_np.shape[1]):
                row = weights_np[head, i, :]
                entropy = -np.sum(row * np.log(row + 1e-9))
                head_entropy.append(entropy)
            entropies.append(np.mean(head_entropy))
        return entropies
    
    entropy_no_pos = calculate_entropy(weights_no_pos)
    entropy_with_pos = calculate_entropy(weights_with_pos)
    
    for head in range(4):
        print(f"头 {head+1}:")
        print(f"  无位置编码平均熵: {entropy_no_pos[head]:.3f}")
        print(f"  有位置编码平均熵: {entropy_with_pos[head]:.3f}")
        print(f"  熵增加: {entropy_with_pos[head] - entropy_no_pos[head]:.3f}")
    
    # 计算对角线注意力强度（自注意力）
    def calculate_diagonal_attention(weights):
        weights_np = weights[0].detach().numpy()
        diagonal_strengths = []
        for head in range(weights_np.shape[0]):
            diagonal = np.diag(weights_np[head])
            diagonal_strengths.append(np.mean(diagonal))
        return diagonal_strengths
    
    diag_no_pos = calculate_diagonal_attention(weights_no_pos)
    diag_with_pos = calculate_diagonal_attention(weights_with_pos)
    
    print("\n对角线注意力强度（自注意力）:")
    for head in range(4):
        print(f"头 {head+1}: 无位置 {diag_no_pos[head]:.3f}, 有位置 {diag_with_pos[head]:.3f}")

# 运行演示
demonstrate_positional_encoding_effect()

## 4.9 位置编码的变体和优化

让我们探索一些位置编码的变体：

In [ ]:
class RoPEPositionalEncoding(nn.Module):
    """
    旋转位置编码 (Rotary Position Embedding - RoPE)
    用于GPT-NeoX、LLaMA等模型
    """
    def __init__(self, d_model: int, max_seq_len: int = 5000):
        super(RoPEPositionalEncoding, self).__init__()
        
        self.d_model = d_model
        
        # 计算频率
        inv_freq = 1.0 / (10000 ** (torch.arange(0, d_model, 2).float() / d_model))
        self.register_buffer('inv_freq', inv_freq)
        
        # 预计算位置编码
        self._update_cos_sin_cache(max_seq_len)
    
    def _update_cos_sin_cache(self, seq_len: int):
        positions = torch.arange(seq_len, dtype=torch.float)
        freqs = torch.outer(positions, self.inv_freq)
        
        cos_freqs = torch.cos(freqs)
        sin_freqs = torch.sin(freqs)
        
        self.register_buffer('cos_cache', cos_freqs)
        self.register_buffer('sin_cache', sin_freqs)
    
    def apply_rope(self, x: torch.Tensor, seq_len: int) -> torch.Tensor:
        """
        应用旋转位置编码
        
        Args:
            x: [batch_size, seq_len, d_model]
        
        Returns:
            rotated_x: [batch_size, seq_len, d_model]
        """
        if seq_len > self.cos_cache.size(0):
            self._update_cos_sin_cache(seq_len)
        
        cos = self.cos_cache[:seq_len]
        sin = self.sin_cache[:seq_len]
        
        # 将输入分为偶数和奇数维度
        x_even = x[..., 0::2]  # 偶数维度
        x_odd = x[..., 1::2]   # 奇数维度
        
        # 应用旋转
        rotated_even = x_even * cos.unsqueeze(0) - x_odd * sin.unsqueeze(0)
        rotated_odd = x_even * sin.unsqueeze(0) + x_odd * cos.unsqueeze(0)
        
        # 重新组合
        rotated_x = torch.zeros_like(x)
        rotated_x[..., 0::2] = rotated_even
        rotated_x[..., 1::2] = rotated_odd
        
        return rotated_x

class ALiBiPositionalEncoding(nn.Module):
    """
    ALiBi (Attention with Linear Biases) 位置编码
    用于PaLM等模型
    """
    def __init__(self, num_heads: int):
        super(ALiBiPositionalEncoding, self).__init__()
        
        self.num_heads = num_heads
        
        # 计算每个头的斜率
        slopes = self._get_slopes(num_heads)
        self.register_buffer('slopes', slopes)
    
    def _get_slopes(self, num_heads: int) -> torch.Tensor:
        """
        计算ALiBi斜率
        """
        def get_slopes_power_of_2(n):
            start = (2**(-2**-(math.log2(n)-3)))
            ratio = start
            return [start*ratio**i for i in range(n)]
        
        if math.log2(num_heads).is_integer():
            slopes = get_slopes_power_of_2(num_heads)
        else:
            closest_power_of_2 = 2**math.floor(math.log2(num_heads))
            slopes = get_slopes_power_of_2(closest_power_of_2)
            slopes.extend(get_slopes_power_of_2(2*closest_power_of_2)[0:num_heads-closest_power_of_2])
        
        return torch.tensor(slopes, dtype=torch.float32)
    
    def forward(self, seq_len: int) -> torch.Tensor:
        """
        计算ALiBi位置偏置
        
        Args:
            seq_len: 序列长度
        
        Returns:
            bias: [num_heads, seq_len, seq_len]
        """
        # 创建距离矩阵
        positions = torch.arange(seq_len, dtype=torch.float32, device=self.slopes.device)
        distances = positions.unsqueeze(1) - positions.unsqueeze(0)
        
        # 应用斜率
        bias = self.slopes.unsqueeze(1).unsqueeze(2) * distances.unsqueeze(0)
        
        return bias

# 测试不同的位置编码变体
d_model = 32
seq_len = 10
num_heads = 4

# 创建测试输入
test_input = torch.randn(1, seq_len, d_model)

# RoPE
rope = RoPEPositionalEncoding(d_model)
rope_output = rope.apply_rope(test_input, seq_len)

# ALiBi
alibi = ALiBiPositionalEncoding(num_heads)
alibi_bias = alibi(seq_len)

print(f"原始输入形状: {test_input.shape}")
print(f"RoPE输出形状: {rope_output.shape}")
print(f"ALiBi偏置形状: {alibi_bias.shape}")

# 可视化ALiBi偏置
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for head in range(num_heads):
    im = axes[head].imshow(alibi_bias[head].numpy(), cmap='RdBu_r')
    axes[head].set_title(f'ALiBi偏置 - 头 {head+1}')
    axes[head].set_xlabel('Key位置')
    axes[head].set_ylabel('Query位置')
    plt.colorbar(im, ax=axes[head])

plt.tight_layout()
plt.show()

print(f"\nALiBi斜率: {alibi.slopes.tolist()}")

## 4.10 位置编码的性能比较

让我们比较不同位置编码方法的性能特点：

In [ ]:
import time

def benchmark_positional_encodings():
    """
    基准测试不同位置编码的性能
    """
    d_model = 512
    seq_len = 1024
    batch_size = 16
    num_heads = 8
    
    # 创建测试数据
    test_data = torch.randn(batch_size, seq_len, d_model)
    
    # 创建不同的位置编码
    encodings = {
        '正弦位置编码': SinusoidalPositionalEncoding(d_model),
        '可学习位置编码': LearnedPositionalEncoding(d_model),
        'RoPE编码': RoPEPositionalEncoding(d_model),
        'ALiBi编码': ALiBiPositionalEncoding(num_heads)
    }
    
    results = {}
    
    print("=== 位置编码性能基准测试 ===")
    print(f"测试配置: batch_size={batch_size}, seq_len={seq_len}, d_model={d_model}")
    print()
    
    for name, encoding in encodings.items():
        # 预热
        for _ in range(5):
            if name == 'RoPE编码':
                _ = encoding.apply_rope(test_data, seq_len)
            elif name == 'ALiBi编码':
                _ = encoding(seq_len)
            else:
                _ = encoding(test_data)
        
        # 正式测试
        num_runs = 100
        start_time = time.time()
        
        for _ in range(num_runs):
            if name == 'RoPE编码':
                result = encoding.apply_rope(test_data, seq_len)
            elif name == 'ALiBi编码':
                result = encoding(seq_len)
            else:
                result = encoding(test_data)
        
        end_time = time.time()
        avg_time = (end_time - start_time) / num_runs * 1000  # 转换为毫秒
        
        # 计算内存使用（参数量）
        if hasattr(encoding, 'parameters'):
            num_params = sum(p.numel() for p in encoding.parameters())
        else:
            num_params = 0
        
        # 计算额外的特性
        trainable = num_params > 0
        
        if name == 'ALiBi编码':
            extrapolation = "优秀"  # ALiBi对长序列外推性好
        elif name == 'RoPE编码':
            extrapolation = "良好"  # RoPE有一定外推性
        elif name == '正弦位置编码':
            extrapolation = "良好"  # 正弦编码有外推性
        else:
            extrapolation = "有限"  # 可学习编码外推性有限
        
        results[name] = {
            'time': avg_time,
            'params': num_params,
            'trainable': trainable,
            'extrapolation': extrapolation
        }
        
        print(f"{name}:")
        print(f"  平均处理时间: {avg_time:.2f} ms")
        print(f"  参数量: {num_params:,}")
        print(f"  可训练: {'是' if trainable else '否'}")
        print(f"  外推能力: {extrapolation}")
        print()
    
    # 创建对比表格
    import pandas as pd
    
    df_data = {
        '方法': list(results.keys()),
        '处理时间(ms)': [results[name]['time'] for name in results.keys()],
        '参数量': [results[name]['params'] for name in results.keys()],
        '可训练': [results[name]['trainable'] for name in results.keys()],
        '外推能力': [results[name]['extrapolation'] for name in results.keys()]
    }
    
    df = pd.DataFrame(df_data)
    print("位置编码方法对比表:")
    print("=" * 50)
    print(df.to_string(index=False))
    
    return results

# 运行基准测试
benchmark_results = benchmark_positional_encodings()

## 总结

在这个教程中，我们深入学习了位置编码的各个方面：

### 核心概念：
1. **位置编码的必要性**：注意力机制本身是位置不变的
2. **正弦位置编码**：使用sin/cos函数创建位置感知的编码
3. **可学习位置编码**：通过训练学习位置表示
4. **相对位置编码**：直接建模位置间的相对关系

### 数学公式（正弦编码）：
$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

### 不同方法的特点：

| 方法 | 参数量 | 外推能力 | 性能开销 | 适用场景 |
|------|--------|----------|----------|----------|
| 正弦编码 | 0 | 良好 | 低 | 通用Transformer |
| 可学习编码 | O(L·d) | 有限 | 低 | 固定长度序列 |
| RoPE | 0 | 良好 | 中等 | 现代LLM |
| ALiBi | 0 | 优秀 | 低 | 长序列应用 |

### 关键洞察：
1. **位置编码直接影响注意力模式**：它改变了模型对不同位置的关注程度
2. **外推能力很重要**：模型需要处理比训练时更长的序列
3. **没有万能的方法**：不同应用场景适合不同的位置编码
4. **相对位置比绝对位置更重要**：现代方法更关注位置间的相对关系

### 实际应用建议：
- **通用任务**：使用正弦位置编码
- **长序列处理**：考虑ALiBi或RoPE
- **固定长度任务**：可学习位置编码可能效果更好
- **现代LLM**：RoPE已成为主流选择

位置编码解决了Transformer架构中的一个关键问题。在下一个教程中，我们将学习如何将所有这些组件组合成完整的**Transformer块**。

### 下一步学习：
- [05-transformer-block.ipynb](05-transformer-block.ipynb) - Transformer基本块详解